In [14]:
import pandas as pd
import numpy as np
import scipy.stats as st

df = pd.read_excel('data.xlsx')
df.COMBUSTIVEL = 'D10)'
df_BA = df[df.ESTADO == 'BAHIA']
df_CE = df[df.ESTADO == 'CEARA']

In [32]:
def analyse_df(df): 
    def get_range_value(value, first=True):
        index = 0 if first else 1
        value = str(value).replace("(", "").replace("]", "").replace(" ", "").split(",")[index]
        return float(value)
        
    def get_prob(value, first=True):
        value = value['Zi(Li)'] if first else value['Zi(Ls)']
        value = st.norm.cdf(value)
        return value
        
    k = int(round(1 + (10/3) * np.log10(df.shape[0])))
    print(f'K: {k}')
    frequency = pd.value_counts(
        pd.cut(x = df.PRE_VENDA, bins = k, include_lowest = True), sort = False)

    ## percentage
    percentage = pd.value_counts(
    pd.cut(x=df.PRE_VENDA, bins = k, include_lowest = True), sort = False, normalize = True) *100 

    ## Formatting Frequency Table
    df_ = ({'O(i)' : frequency, 'pi' : percentage/100, 'Pi': percentage.cumsum()/100})

    df_ = pd.DataFrame(df_)
    df_.reset_index(inplace=True)
    df_.rename(columns={'index' : 'range'}, inplace = True)
    
    df_['Xi'] = df_.range.apply(lambda range: range.mid).astype(np.float32)
    
    df_['Xi*pi'] = df_.Xi.mul(df_.pi)
    media_DA = df_['Xi*pi'].sum()
    
    df_['di^2*pi'] = df_.apply(lambda row: ((row.Xi - media_DA)**2) * row.pi, axis=1)
    var_DA = df_['di^2*pi'].sum()
    desvio_DA = np.sqrt(df_['di^2*pi'].sum())
    
    # df_['Zi(Li)'] = (primeiro valor da classe - media_DA) / desvio_DA
    # df_['Zi(Ls)'] = (primeiro valor da classe - media_DA) / desvio_DA
    # df_['E(i)'] = (Pr Ls - Pr Li) * df.shape[0]

    df_['Zi(Li)'] = df_.apply(lambda row: (get_range_value(row.range) - media_DA) / desvio_DA, axis=1)
    df_['Zi(Ls)'] = df_.apply(lambda row: (get_range_value(row.range, False) - media_DA) / desvio_DA, axis=1)
    df_['Pr Ac até Li'] = df_.apply(lambda row: get_prob(row), axis=1)
    df_['Pr Ac até Ls'] = df_.apply(lambda row: get_prob(row, False), axis=1)


    df_['E(i)'] = (df_['Pr Ac até Ls'] - df_['Pr Ac até Li']) * df.shape[0]
    df_['(O(i)-E(i))ˆ2/E(i)'] = df_.apply(lambda row: pow((row['O(i)'] - row['E(i)']), 2)/row['E(i)'], axis=1)
    
    
    media_DNA = df.PRE_VENDA.mean()
    mediana_DNA = df.PRE_VENDA.median()
    desvio_DNA = df.PRE_VENDA.std()
    var_DNA = df.PRE_VENDA.std()**2
    
    erro_media = abs(media_DNA - media_DA) * 100 / media_DNA
    erro_desvio = abs(desvio_DNA - desvio_DA) * 100 / desvio_DNA
    erro_var = abs(var_DNA - var_DA) * 100 / var_DNA

    CV = desvio_DA / media_DA
    print(f'Medias: DNA: {media_DNA} DA: {media_DA}')
    print(f'Mediana: DNA: {mediana_DNA} DA: ')
    print(f'Desvios: DNA: {desvio_DNA} DA: {desvio_DA}')
    print(f'Variância: DNA: {var_DNA} DA: {var_DA}')
    print(f'CV: {CV}')
    print(f'Erro da média: {erro_media}')
    print(f'Erro do desvio padrão: {erro_desvio}')
    print(f'Erro da variância: {erro_var}')
    print(f'Razao: {var_DA ** 2}')
    print(f'Razao: {df.shape[0]}')
    return df_

In [33]:
# Analise BAHIA
analyse_df(df_BA).to_excel('output.xlsx')

K: 8
Medias: DNA: 4.6686193181818165 DA: 4.665525563738563
Mediana: DNA: 4.655 DA: 
Desvios: DNA: 0.16575340538312897 DA: 0.16832005259851693
Variância: DNA: 0.027474191396103888 DA: 0.02833164010676751
CV: 0.036077404420787114
Erro da média: 0.06626701027442722
Erro do desvio padrão: 1.5484732934779306
Erro da variância: 3.1209242823620156
Razao: 0.0008026818311393973
Razao: 176


In [34]:
# Analise CEARA
analyse_df(df_CE).to_excel('output.xlsx')

K: 8
Medias: DNA: 4.701899408284025 DA: 4.704500091146436
Mediana: DNA: 4.699 DA: 
Desvios: DNA: 0.1352729007533261 DA: 0.13574187585671013
Variância: DNA: 0.018298757678219212 DA: 0.018425856861098502
CV: 0.028853623812691076
Erro da média: 0.05531132498981851
Erro do desvio padrão: 0.34668813988045033
Erro da variância: 0.694578206424225
Razao: 0.0003395122010656908
Razao: 169
